In [3]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from numpy import loadtxt

In [5]:
tr = loadtxt("dataset/monk1/monks-1.train", dtype='str', delimiter=' ')
tr = np.delete(tr, [0, 8], 1)
tr[:10]

array([['1', '1', '1', '1', '1', '3', '1'],
       ['1', '1', '1', '1', '1', '3', '2'],
       ['1', '1', '1', '1', '3', '2', '1'],
       ['1', '1', '1', '1', '3', '3', '2'],
       ['1', '1', '1', '2', '1', '2', '1'],
       ['1', '1', '1', '2', '1', '2', '2'],
       ['1', '1', '1', '2', '2', '3', '1'],
       ['1', '1', '1', '2', '2', '4', '1'],
       ['1', '1', '1', '2', '3', '1', '2'],
       ['1', '1', '2', '1', '1', '1', '2']], dtype='<U8')

In [6]:
tr = tr.astype(int)
tr[:20]

array([[1, 1, 1, 1, 1, 3, 1],
       [1, 1, 1, 1, 1, 3, 2],
       [1, 1, 1, 1, 3, 2, 1],
       [1, 1, 1, 1, 3, 3, 2],
       [1, 1, 1, 2, 1, 2, 1],
       [1, 1, 1, 2, 1, 2, 2],
       [1, 1, 1, 2, 2, 3, 1],
       [1, 1, 1, 2, 2, 4, 1],
       [1, 1, 1, 2, 3, 1, 2],
       [1, 1, 2, 1, 1, 1, 2],
       [0, 1, 2, 1, 1, 2, 1],
       [0, 1, 2, 1, 1, 3, 1],
       [0, 1, 2, 1, 1, 4, 2],
       [1, 1, 2, 1, 2, 1, 1],
       [0, 1, 2, 1, 2, 3, 1],
       [0, 1, 2, 1, 2, 3, 2],
       [0, 1, 2, 1, 2, 4, 2],
       [0, 1, 2, 1, 3, 2, 1],
       [0, 1, 2, 1, 3, 4, 2],
       [0, 1, 2, 2, 1, 2, 2]])

In [7]:
tr_y = tr[:, [0]]
tr_x = tr[:, range(1,7)]

In [8]:
tr_x[:20]

array([[1, 1, 1, 1, 3, 1],
       [1, 1, 1, 1, 3, 2],
       [1, 1, 1, 3, 2, 1],
       [1, 1, 1, 3, 3, 2],
       [1, 1, 2, 1, 2, 1],
       [1, 1, 2, 1, 2, 2],
       [1, 1, 2, 2, 3, 1],
       [1, 1, 2, 2, 4, 1],
       [1, 1, 2, 3, 1, 2],
       [1, 2, 1, 1, 1, 2],
       [1, 2, 1, 1, 2, 1],
       [1, 2, 1, 1, 3, 1],
       [1, 2, 1, 1, 4, 2],
       [1, 2, 1, 2, 1, 1],
       [1, 2, 1, 2, 3, 1],
       [1, 2, 1, 2, 3, 2],
       [1, 2, 1, 2, 4, 2],
       [1, 2, 1, 3, 2, 1],
       [1, 2, 1, 3, 4, 2],
       [1, 2, 2, 1, 2, 2]])

In [9]:
range(6)

range(0, 6)